In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
 

In [16]:
df["Mark"] = df["G3"].apply(lambda x: "A" if x >=17 else "B" if x >= 15 else "C" if x >= 12 else "D" if x >= 11 else "F" )
df.drop("G3",axis=1)
X,y = df.drop("Mark",axis=1),df["Mark"]